# Europe

**Source: [https://digital-agenda-data.eu/](https://digital-agenda-data.eu/)**

### Key indicators used:
1) Broadband take-up and coverage
- Standard fixed broadband coverage/availability (bb_scov)
- NGA broadband coverage/availability (bb_ngacov)
- Households with a broadband connection (h_broad)
- Households with fixed broadband connection (h_bbfix)

2) Broadband speeds and prices
- Share of fixed broadband subscriptions >= 100 Mbps (bb_speed100)
- Households with no access to Internet at home, because the costs are too high (all households) (h_xcost)

3) Mobile market
- 4G mobile broadband coverage (total) (mbb_ltecov)

4) Internet usage
- Households with access to the Internet at home (h_iacc)
- Individuals who have never used the internet (all individuals) (i_iux)

5) Take up of internet services (all individuals)
- Using online banking (i_iubk)
- Uploading self-created content to be shared (i_iuupl)
- Participating in social networks, over the internet, last 3 months (i_iusnet)
- Looking for a job or sending a job application (i_iujob)
- Doing an online course (i_iuolc)
- Taking part in on-line consultations or voting to define civic or political issues (i_iuvote)

6) eGovernment
- Individuals interacting online with public authorities, last 12 months (i_iugov12)

7) Digital skills
- Individuals with at least basic digital skills (i_dsk_bab)

8) ICT Specialists
- Individuals who have obtained ICT skills through formal educational institutions (i_skedu)

## Import libraries

In [1]:
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from ipywidgets import widgets
import numpy as np 
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

## Preprocessing

In [42]:
key_indicators = pd.read_csv("data/digital-agenda-scoreboard-key-indicators.csv")
# Drop useless columns
key_indicators = key_indicators.drop(['observation', 'flag', 'note'], axis=1)
# Filter out some indicators
my_indicators = ['bb_scov', 'bb_ngacov', 'h_broad', 'h_bbfix', 'bb_speed100', 'h_xcost', 'mbb_ltecov', 'h_iacc', 'i_iux', 'i_iubk', 'i_iuupl', 'i_iusnet', 'i_iujob', 'i_iuolc', 'i_iuvote', 'i_iugov12', 'i_dsk_s_bab', 'i_skedu']
key_indicators = key_indicators[key_indicators['indicator'].isin(my_indicators)]
# Map the indicators' codes to the corresponding name
indicators_mapping = {'bb_scov' : 'Standard fixed broadband coverage/availability',
           "bb_ngacov" : 'NGA broadband coverage/availability',
           'h_broad' : 'Households with a broadband connection',
           'h_bbfix' : 'Households with fixed broadband connection',
           'bb_speed100' : 'Share of fixed broadband subscriptions >= 100 Mbps',
           'h_xcost' : 'Households with no access to Internet at home, because the costs are too high',
           'mbb_ltecov' : '4G mobile broadband coverage',
           'h_iacc' : 'Households with access to the Internet at home',
           'i_iux' : 'Individuals who have never used the internet',
           'i_iubk' : 'Individuals using online banking',
           'i_iuupl' : 'Individuals uploading self-created content to be shared',
           'i_iusnet' : 'Individuals participating in social networks, over the internet, last 3 months',
           'i_iujob' : 'Individuals looking for a job or sending a job application',
           'i_iuolc' : 'Individuals doing an online course',
           'i_iuvote' : 'Individuals taking part in on-line consultations or voting to define civic or political issues',
           'i_iugov12' : 'Individuals interacting online with public authorities, last 12 months',
           'i_dsk_s_bab' : 'Individuals with at least basic digital skills in Software domain',
           'i_skedu' : 'Individuals who have obtained ICT skills through formal educational institutions',
          }
key_indicators['indicator'] = key_indicators['indicator'].replace(indicators_mapping)

# Associate the indicators with their corresponding number (i.e. the group)
groups = {'1' : ['Standard fixed broadband coverage/availability',
          'NGA broadband coverage/availability',
          'Households with a broadband connection',
          'Households with fixed broadband connection'],
          '2' : ['Share of fixed broadband subscriptions >= 100 Mbps',
          'Households with no access to Internet at home, because the costs are too high'],
          '3' : ['4G mobile broadband coverage'],
          '4' : ['Households with access to the Internet at home',
          'Individuals who have never used the internet'],
          '5' : ['Individuals using online banking',
          'Individuals uploading self-created content to be shared',
          'Individuals participating in social networks, over the internet, last 3 months',
          'Individuals looking for a job or sending a job application',
          'Individuals doing an online course',
          'Individuals taking part in on-line consultations or voting to define civic or political issues'],
          '6' : ['Individuals interacting online with public authorities, last 12 months'],
          '7' : ['Individuals with at least basic digital skills in Software domain'],
          '8' : ['Individuals who have obtained ICT skills through formal educational institutions']
          }
    
# Mapping between 'ref_area' and the name of the country
countries_mapping =  { 'EE' : 'Estonia',
                        'EL' : 'Greece',
                        'ES' : 'Spain',
                        'FI' : 'Finland',
                        'FR' : 'France',
                        'HR' : 'Croatia',
                        'HU' : 'Hungary',
                        'IE' : 'Ireland',
                        'IS' : 'Iceland',
                        'DE' : 'Deutschland',
                        'CZ' : 'Czech Rep.',
                        'DK' : 'Denmark',
                        'IT' : 'Italy',
                        'LT' : 'Lithuania',
                        'LU' : 'Luxembourg',
                        'LV' : 'Latvia',
                        'MT' : 'Malta',
                        'NL' : 'Netherlands',
                        'NO' : 'Norway',
                        'PT' : 'Portugal',
                        'PL' : 'Poland',
                        'EU' : 'Europe',
                        'RO' : 'Romania',
                        'SE' : 'Sweden',
                        'SI' : 'Slovenia',
                        'SK' : 'Slovakia',
                        'UK' : 'United Kingdom',
                        'AT' : 'Austria',
                        'BE' : 'Belgium',
                        'BG' : 'Bulgaria',
                        'CY' : 'Cyprus',
                        'CH' : 'Switzerland'
                     }
key_indicators['ref_area'] = key_indicators['ref_area'].replace(countries_mapping)

# Drop the index column
key_indicators.reset_index(drop=True, inplace=True)

# In some cases the types in 'time_period' are mixed ==> cast all to strings 
key_indicators['time_period'] = key_indicators['time_period'].apply(lambda j: str(j))
def convert_to_date(j):
   if len(j.split('-')) == 1:
      return str(datetime.strptime(j, '%Y'))
   elif len(j.split('-')) == 2:
      return str(datetime.strptime(j, '%Y-%m'))
   else:
      return str(datetime.strptime(j, '%Y-%m-%d'))
key_indicators['time_period'] = key_indicators['time_period'].apply(lambda j: convert_to_date(j))

In [39]:
def generate_rgb_colors():
    palette = sns.color_palette('Paired') + sns.color_palette('Spectral') + sns.color_palette('bright') + sns.color_palette('Dark2')
    return palette

options = ['Broadband take-up and coverage', 'Broadband speeds and prices', 'Mobile market', 'Internet usage', 'Take up of internet services', 'eGovernment', 'Digital skills', 'ICT Specialists']
countries_list = [country for country in countries_mapping.values()]

indicators_group = widgets.Dropdown(options=options, value='Broadband take-up and coverage')
# Get the group selected in the first dropdown
selected_group = indicators_group.value
# Retrieve the index from the 'options' list and the corresponding indicators
index = options.index(selected_group) + 1
corresponding_indicators = groups.get(str(index))
# Put the retrieved indicators in the second dropdown
indicators = widgets.Dropdown(options=corresponding_indicators)

box_layout = widgets.Layout(margin='0px 0px 0px 80px')
title_and_description = widgets.VBox([widgets.HTML('<h1>See the evolution of an indicator and compare countries</h1>'), widgets.HTML('<p>Click on the name of the countries in the legend in order to visualize the corresponding values in the graph.</p>')])
controllers = widgets.HBox([widgets.Label('Indicator group:'), indicators_group, widgets.Label('Indicator:'), indicators])
introduction = widgets.VBox([title_and_description, controllers], layout=box_layout)

output = widgets.Output()

@output.capture()
def callback(change):
    selected_group = indicators_group.value
    index = options.index(selected_group) + 1
    corresponding_indicators = groups.get(str(index))
    indicators.options = corresponding_indicators
# When the first dropdown changes, change the options in the second one
indicators_group.observe(callback, 'value')

# Callback to update the time series graph when the indicator changes
@output.capture()
def update_graph(change):
    current_indicator = str(indicators.value)
    fig.update_layout(title=current_indicator)
    for index, country in enumerate(fig.data):
        current_data = key_indicators.loc[(key_indicators['indicator'] == current_indicator) & (key_indicators['ref_area'] == country.name)]
        # Check if there are multiple breakdowns: if so, keep only the 'total' one. Do the same for the 'unit_measure'.
        # Note that this is not a general approach!
        if len(current_data['breakdown'].unique()) > 1:
            if len(current_data['unit_measure'].unique()) > 1:
                current_data = current_data.loc[(current_data['breakdown'].str.contains('total')) & (current_data['unit_measure'] == 'pc_ind')]
            else:
                current_data = current_data.loc[(current_data['breakdown'].str.contains('total'))]
        current_data = current_data.sort_values('time_period')
        # Update the data in the graph
        country.x = list(current_data['time_period'])
        country.y = list(current_data['value'])

indicators.observe(update_graph, 'value')

# GET DATA FROM THE DF 
current_group = str(indicators_group.value)
current_indicator = str(indicators.value)

# Build Figure
fig = go.FigureWidget()
palette = generate_rgb_colors() # Obtain n different hex colors, where n is the number of countries
for index, country in enumerate(countries_list):
    current_data = key_indicators.loc[(key_indicators['indicator'] == current_indicator) & (key_indicators['ref_area'] == country)]
    # Check if there are multiple breakdowns: if so, keep only the 'total' one
    if len(current_data['breakdown'].unique()) > 1:
        current_data = current_data.loc[current_data['breakdown'].str.contains('total')]
    current_data = current_data.sort_values('time_period')
    if (country == 'Italy') or (country == 'Europe'):
        visible = True
    else:
        visible = 'legendonly'
    fig.add_trace(go.Scatter(
        x=list(current_data['time_period']),
        y=list(current_data['value']),
        name=country,
        visible=visible,
        mode='lines+markers+text',
        textposition='bottom center',
        text=country,
        line=dict(color='rgb' + str(palette[index]), shape='spline', smoothing=1.3)))

fig.update_layout(height=600, title=current_indicator,
                xaxis_title='Year',
                yaxis_title='Percentage',
                legend_title_text='Countries',
                showlegend=True)

widgets.VBox([introduction, fig])

## Experiment

In [44]:
# Since some values in 'value' column are strings, convert them into float
key_indicators["value"] = key_indicators["value"].astype(float)
# Multiply by 100 to have a percentage
key_indicators['value'] = key_indicators['value'] * 100
key_indicators.head()

,time_period,ref_area,indicator,breakdown,unit_measure,value
0,2019-01-01 00:00:00,Estonia,"Individuals participating in social networks, ...",cb_eu_for,pc_ind,69.9007
1,2019-01-01 00:00:00,Estonia,"Individuals participating in social networks, ...",cb_eu_for,pc_ind_iu3,75.9500
2,2019-01-01 00:00:00,Estonia,Individuals uploading self-created content to ...,cb_eu_for,pc_ind,45.1348
3,2019-01-01 00:00:00,Estonia,Individuals uploading self-created content to ...,cb_eu_for,pc_ind_iu3,49.0408
4,2019-01-01 00:00:00,Estonia,Individuals taking part in on-line consultatio...,cb_eu_for,pc_ind,19.3373


In [46]:
 labels = ['Standard fixed broadband coverage/availability',
          'NGA broadband coverage/availability',
          'Households with a broadband connection',
          'Households with fixed broadband connection',
          'Share of fixed broadband subscriptions >= 100 Mbps',
          'Households with no access to Internet at home, because the costs are too high',
          '4G mobile broadband coverage',
          'Households with access to the Internet at home',
          'Individuals who have never used the internet',
          'Individuals using online banking',
          'Individuals uploading self-created content to be shared',
          'Individuals participating in social networks, over the internet, last 3 months',
          'Individuals looking for a job or sending a job application',
          'Individuals doing an online course',
          'Individuals taking part in on-line consultations or voting to define civic or political issues',
          'Individuals interacting online with public authorities, last 12 months',
          'Individuals with at least basic digital skills in Software domain',
          'Individuals who have obtained ICT skills through formal educational institutions']

In [47]:
countries_list = [country for country in countries_mapping.values()]

# Get the x,y data of all the countries given a label
def get_data(label):
    x_data = []
    y_data = []
    for country in countries_list:
        data = key_indicators.loc[(key_indicators['indicator'] == label) & (key_indicators['ref_area'] == country)]
        # Check if there are multiple breakdowns: if so, keep only the 'total' one
        if len(data['breakdown'].unique()) > 1:
            if len(data['unit_measure'].unique()) > 1:
                data = data.loc[(data['breakdown'].str.contains('total')) & (data['unit_measure'] == 'pc_ind')]
            else:
                data = data.loc[(data['breakdown'].str.contains('total'))]
        data = data.sort_values('time_period')
        x_data.append(list(data['time_period']))
        y_data.append(list(data['value']))
    return x_data, y_data

current_indicator = labels[0]

# Build Figure
fig = go.FigureWidget()
palette = generate_rgb_colors() # Obtain n different hex colors, where n is the number of countries

for index, country in enumerate(countries_list):
    current_data = key_indicators.loc[(key_indicators['indicator'] == current_indicator) & (key_indicators['ref_area'] == country)]
    # Check if there are multiple breakdowns: if so, keep only the 'total' one
    if len(current_data['breakdown'].unique()) > 1:
        current_data = current_data.loc[current_data['breakdown'].str.contains('total')]
    current_data = current_data.sort_values('time_period')
    if (country == 'Italy') or (country == 'Europe'):
        visible = True
    else:
        visible = 'legendonly'
    fig.add_trace(go.Scatter(
        x=list(current_data['time_period']),
        y=list(current_data['value']),
        name=country,
        visible=visible,
        mode='lines+markers+text',
        textposition='bottom center',
        text=country,
        line=dict(color='rgb' + str(palette[index]), shape='spline', smoothing=1.0)))

# Create one button for each label
buttons = []
for index, label in enumerate(labels):
    x_data, y_data = get_data(label)
    btn = dict(label=label, method='update', args=[{'x': x_data, 'y': y_data}])
    buttons.append(btn)


fig.update_layout(width=1200, height=600, title='Select an indicator and compare countries',
                  xaxis_title='Year',
                  yaxis_title='Percentage',
                  legend_title_text='Countries',
                  showlegend=True,
                  updatemenus=[
                    dict(active=0, buttons=buttons, xanchor='right', yanchor='top', showactive=True, x=1.1, y=1.17)
                  ])

fig.update_yaxes(rangemode="tozero")

fig.show()

In [48]:
fig.write_html("time_series.html")